In [1]:
import pandas as pd
from holidays import UnitedStates
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import calendar
import scipy.stats as stats
from scipy.stats import spearmanr
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pingouin as pg
from pandas.plotting import autocorrelation_plot
import statsmodels.api as sm
from sklearn.metrics import roc_curve, auc,precision_score, recall_score, f1_score, mean_squared_error, classification_report
from statsmodels.tsa.arima.model import ARIMA
import os


# Datasets Reading

In [2]:
taxi_df = pd.read_csv("Datasets Folder/dataset.csv", index_col=0)
# Create a new row for 2015-02-01 00:00:00
new_row = {'timestamp': pd.Timestamp('2015-02-01 00:00:00'), 'value': np.nan}

# Append the new row to the original DataFrame
taxi_df.loc[len(taxi_df)] = new_row
taxi_df

,timestamp,value
0,2014-07-01 00:00:00,10844.0
1,2014-07-01 00:30:00,8127.0
2,2014-07-01 01:00:00,6210.0
3,2014-07-01 01:30:00,4656.0
4,2014-07-01 02:00:00,3820.0
...,...,...
10316,2015-01-31 22:00:00,25721.0
10317,2015-01-31 22:30:00,27309.0
10318,2015-01-31 23:00:00,26591.0
10319,2015-01-31 23:30:00,26288.0


In [3]:
print(taxi_df.isnull().sum())
taxi_df.info()

timestamp    0
value        1
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 10321 entries, 0 to 10320
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  10321 non-null  object 
 1   value      10320 non-null  float64
dtypes: float64(1), object(1)
memory usage: 241.9+ KB


In [4]:
weather_df = pd.read_csv('Datasets Folder/weather_csv.csv')
weather_df.head()

,timestamp,prep,snow,snow_depth,min_temp,max_temp
0,2014-07-01,0.00,0.0,0.0,72.0,89.0
1,2014-07-02,0.96,0.0,0.0,72.0,91.0
2,2014-07-03,1.78,0.0,0.0,69.0,87.0
3,2014-07-04,0.14,0.0,0.0,65.0,74.0
4,2014-07-05,0.00,0.0,0.0,63.0,81.0


In [5]:
print(weather_df.isnull().sum())
weather_df.info()

timestamp     0
prep          0
snow          0
snow_depth    0
min_temp      0
max_temp      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   timestamp   216 non-null    object 
 1   prep        216 non-null    float64
 2   snow        216 non-null    float64
 3   snow_depth  216 non-null    float64
 4   min_temp    216 non-null    float64
 5   max_temp    216 non-null    float64
dtypes: float64(5), object(1)
memory usage: 10.3+ KB


In [6]:
# Create a holiday object for the US with state-specific holidays for New York
ny_holidays = UnitedStates(years=[2014, 2015], state='NY')

# Convert to a DataFrame
holidays_df = pd.DataFrame({
    "Date": list(ny_holidays.keys()),
    "Holiday": list(ny_holidays.values())
})

# Convert 'Date' to datetime
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])

# Filter holidays up to '2015-01-31'
holidays_df = holidays_df.loc[(holidays_df['Date'] <= '2015-01-31') & (holidays_df['Date'] >= '2014-07-01')]

# Sort by index and rename the column "Date" to "timestamp"
holidays_df = holidays_df.sort_values(by=['Date'])

# Show the final DataFrame
holidays_df

,Date,Holiday
2,2014-07-04,Independence Day
3,2014-09-01,Labor Day
9,2014-10-13,Columbus Day
12,2014-11-04,Election Day
4,2014-11-11,Veterans Day
5,2014-11-27,Thanksgiving
6,2014-12-25,Christmas Day
13,2015-01-01,New Year's Day
21,2015-01-19,Martin Luther King Jr. Day


In [7]:
taxi_df['timestamp'] = pd.to_datetime(taxi_df['timestamp'])
taxi_df['year'] = taxi_df['timestamp'].dt.year
taxi_df['month'] = taxi_df['timestamp'].dt.month
taxi_df['day'] = taxi_df['timestamp'].dt.day
taxi_df['Date'] = taxi_df['timestamp'].dt.date
taxi_df['Date'] = taxi_df['Date'].apply(pd.to_datetime).dt.date
taxi_df['day_of_week'] = taxi_df['timestamp'].dt.day_of_week + 1
taxi_df['hour'] = taxi_df['timestamp'].dt.hour
taxi_df['minute'] = taxi_df['timestamp'].dt.minute
taxi_df['isWeekend'] = taxi_df.day_of_week.map({1: False, 2: False, 3: False, 4: False, 5: False, 6: True, 7: True})
taxi_df['day_of_week'] = taxi_df['day_of_week'].map({1: 'Monday', 2:'Tuesday'
                                                     , 3:'Wednesday', 4:"Thursday", 5: 'Friday',
                                                     6: 'Saturday', 7: 'Sunday'})

# Add time parts to the anomaly dates and update the 'isAnomaly' column
taxi_df['isAnomaly'] = False
anomaly_dates = ['2014-11-02', '2014-11-27', '2014-12-24', '2014-12-25', '2015-01-01', '2015-01-26', '2015-01-27']

anomaly_dates = [pd.to_datetime(date).date() for date in anomaly_dates]
taxi_df['isAnomaly'] = taxi_df['Date'].isin(anomaly_dates)

# View the dataset for verification
taxi_df.loc[taxi_df['isAnomaly']][['Date', 'isAnomaly']]

,Date,isAnomaly
5952,2014-11-02,True
5953,2014-11-02,True
5954,2014-11-02,True
5955,2014-11-02,True
5956,2014-11-02,True
...,...,...
10123,2015-01-27,True
10124,2015-01-27,True
10125,2015-01-27,True
10126,2015-01-27,True


In [8]:
taxi_df

,timestamp,value,year,month,day,Date,day_of_week,hour,minute,isWeekend,isAnomaly
0,2014-07-01 00:00:00,10844.0,2014,7,1,2014-07-01,Tuesday,0,0,False,False
1,2014-07-01 00:30:00,8127.0,2014,7,1,2014-07-01,Tuesday,0,30,False,False
2,2014-07-01 01:00:00,6210.0,2014,7,1,2014-07-01,Tuesday,1,0,False,False
3,2014-07-01 01:30:00,4656.0,2014,7,1,2014-07-01,Tuesday,1,30,False,False
4,2014-07-01 02:00:00,3820.0,2014,7,1,2014-07-01,Tuesday,2,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...
10316,2015-01-31 22:00:00,25721.0,2015,1,31,2015-01-31,Saturday,22,0,True,False
10317,2015-01-31 22:30:00,27309.0,2015,1,31,2015-01-31,Saturday,22,30,True,False
10318,2015-01-31 23:00:00,26591.0,2015,1,31,2015-01-31,Saturday,23,0,True,False
10319,2015-01-31 23:30:00,26288.0,2015,1,31,2015-01-31,Saturday,23,30,True,False


Merging datasets

In [9]:
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])
taxi_df['Date'] = pd.to_datetime(taxi_df['Date'])
merged_data = pd.merge(taxi_df, holidays_df, how='left', on='Date')

weather_df['timestamp'] = weather_df['timestamp'] + ' 00:00:00'
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'] + ' 00:00:00')
weather_df['timestamp'] = weather_df['timestamp'].dt.normalize()
weather_df
merged_data = pd.merge(merged_data, weather_df, how='left', on='timestamp')
merged_data.loc[~merged_data['Holiday'].isnull()]

C:\Users\weare\AppData\Local\Temp\ipykernel_11708\2614963403.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'] + ' 00:00:00')


,timestamp,value,year,month,day,Date,day_of_week,hour,minute,isWeekend,isAnomaly,Holiday,prep,snow,snow_depth,min_temp,max_temp
144,2014-07-04 00:00:00,15591.0,2014,7,4,2014-07-04,Friday,0,0,False,False,Independence Day,0.14,0.0,0.0,65.0,74.0
145,2014-07-04 00:30:00,14395.0,2014,7,4,2014-07-04,Friday,0,30,False,False,Independence Day,NaN,NaN,NaN,NaN,NaN
146,2014-07-04 01:00:00,12535.0,2014,7,4,2014-07-04,Friday,1,0,False,False,Independence Day,NaN,NaN,NaN,NaN,NaN
147,2014-07-04 01:30:00,11341.0,2014,7,4,2014-07-04,Friday,1,30,False,False,Independence Day,NaN,NaN,NaN,NaN,NaN
148,2014-07-04 02:00:00,9980.0,2014,7,4,2014-07-04,Friday,2,0,False,False,Independence Day,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9739,2015-01-19 21:30:00,14069.0,2015,1,19,2015-01-19,Monday,21,30,False,False,Martin Luther King Jr. Day,NaN,NaN,NaN,NaN,NaN
9740,2015-01-19 22:00:00,13506.0,2015,1,19,2015-01-19,Monday,22,0,False,False,Martin Luther King Jr. Day,NaN,NaN,NaN,NaN,NaN
9741,2015-01-19 22:30:00,12936.0,2015,1,19,2015-01-19,Monday,22,30,False,False,Martin Luther King Jr. Day,NaN,NaN,NaN,NaN,NaN
9742,2015-01-19 23:00:00,10400.0,2015,1,19,2015-01-19,Monday,23,0,False,False,Martin Luther King Jr. Day,NaN,NaN,NaN,NaN,NaN


In [10]:
dataset = merged_data.copy()
dataset

,timestamp,value,year,month,day,Date,day_of_week,hour,minute,isWeekend,isAnomaly,Holiday,prep,snow,snow_depth,min_temp,max_temp
0,2014-07-01 00:00:00,10844.0,2014,7,1,2014-07-01,Tuesday,0,0,False,False,NaN,0.00,0.0,0.0,72.0,89.0
1,2014-07-01 00:30:00,8127.0,2014,7,1,2014-07-01,Tuesday,0,30,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-07-01 01:00:00,6210.0,2014,7,1,2014-07-01,Tuesday,1,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-07-01 01:30:00,4656.0,2014,7,1,2014-07-01,Tuesday,1,30,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-07-01 02:00:00,3820.0,2014,7,1,2014-07-01,Tuesday,2,0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10316,2015-01-31 22:00:00,25721.0,2015,1,31,2015-01-31,Saturday,22,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN
10317,2015-01-31 22:30:00,27309.0,2015,1,31,2015-01-31,Saturday,22,30,True,False,NaN,NaN,NaN,NaN,NaN,NaN
10318,2015-01-31 23:00:00,26591.0,2015,1,31,2015-01-31,Saturday,23,0,True,False,NaN,NaN,NaN,NaN,NaN,NaN
10319,2015-01-31 23:30:00,26288.0,2015,1,31,2015-01-31,Saturday,23,30,True,False,NaN,NaN,NaN,NaN,NaN,NaN


Filling the missing values

In [11]:
dataset = dataset[['timestamp','value','month','day','hour','minute','day_of_week','isWeekend','isAnomaly','Holiday','prep','snow','snow_depth','min_temp','max_temp']]
dataset['isHoliday'] = False
dataset.loc[~dataset['Holiday'].isnull(),'isHoliday'] = True
dataset['TotalMinutes'] = dataset['minute'] + dataset['hour'] * 60
dataset['season'] = dataset['month'].map({7: 'Summer', 8:'Summer', 9:"Autumn",
                                        10:'Autumn', 11:"Autumn", 12: 'Winter',
                                                     1: 'Winter'})
dataset

C:\Users\weare\AppData\Local\Temp\ipykernel_11708\843231166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['isHoliday'] = False


,timestamp,value,month,day,hour,minute,day_of_week,isWeekend,isAnomaly,Holiday,prep,snow,snow_depth,min_temp,max_temp,isHoliday,TotalMinutes,season
0,2014-07-01 00:00:00,10844.0,7,1,0,0,Tuesday,False,False,NaN,0.00,0.0,0.0,72.0,89.0,False,0,Summer
1,2014-07-01 00:30:00,8127.0,7,1,0,30,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,30,Summer
2,2014-07-01 01:00:00,6210.0,7,1,1,0,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,60,Summer
3,2014-07-01 01:30:00,4656.0,7,1,1,30,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,90,Summer
4,2014-07-01 02:00:00,3820.0,7,1,2,0,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,120,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10316,2015-01-31 22:00:00,25721.0,1,31,22,0,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1320,Winter
10317,2015-01-31 22:30:00,27309.0,1,31,22,30,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1350,Winter
10318,2015-01-31 23:00:00,26591.0,1,31,23,0,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1380,Winter
10319,2015-01-31 23:30:00,26288.0,1,31,23,30,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1410,Winter


In [12]:
# Ensure the folder exists
# Define the folder name
folder_name = "Datasets Folder"
os.makedirs(folder_name, exist_ok=True)

# Define the file path
file_path = os.path.join(folder_name, "dataset_updated.csv")

# Save the DataFrame as a CSV file
dataset.to_csv(file_path, index=False)

print(f"DataFrame saved successfully to {file_path}")

DataFrame saved successfully to Datasets Folder\dataset_updated.csv


In [13]:
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset.set_index("timestamp", inplace=True)
print(dataset.isnull().sum())
print()
dataset.describe()

value               1
month               0
day                 0
hour                0
minute              0
day_of_week         0
isWeekend           0
isAnomaly           0
Holiday          9889
prep            10105
snow            10105
snow_depth      10105
min_temp        10105
max_temp        10105
isHoliday           0
TotalMinutes        0
season              1
dtype: int64



,value,month,day,hour,minute,prep,snow,snow_depth,min_temp,max_temp,TotalMinutes
count,10320.000000,10321.000000,10321.000000,10321.000000,10321.000000,216.000000,216.000000,216.000000,216.000000,216.000000,10321.000000
mean,15137.569380,8.269160,15.859025,11.498886,14.998547,0.141806,0.085185,0.248611,49.777778,62.995370,704.931693
std,6939.495808,3.379261,8.868658,6.923112,15.000727,0.366058,0.552005,1.191598,17.416823,19.140484,415.659905
min,8.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,21.000000,0.000000
25%,10262.000000,7.000000,8.000000,5.000000,0.000000,0.000000,0.000000,0.000000,35.000000,44.750000,330.000000
50%,16778.000000,9.000000,16.000000,11.000000,0.000000,0.000000,0.000000,0.000000,52.500000,67.000000,690.000000
75%,19838.750000,11.000000,24.000000,17.000000,30.000000,0.060000,0.000000,0.000000,65.250000,81.000000,1050.000000
max,39197.000000,12.000000,31.000000,23.000000,30.000000,2.540000,5.500000,7.900000,77.000000,92.000000,1410.000000


In [14]:
dataset

,value,month,day,hour,minute,day_of_week,isWeekend,isAnomaly,Holiday,prep,snow,snow_depth,min_temp,max_temp,isHoliday,TotalMinutes,season
timestamp,,,,,,,,,,,,,,,,,
2014-07-01 00:00:00,10844.0,7,1,0,0,Tuesday,False,False,NaN,0.00,0.0,0.0,72.0,89.0,False,0,Summer
2014-07-01 00:30:00,8127.0,7,1,0,30,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,30,Summer
2014-07-01 01:00:00,6210.0,7,1,1,0,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,60,Summer
2014-07-01 01:30:00,4656.0,7,1,1,30,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,90,Summer
2014-07-01 02:00:00,3820.0,7,1,2,0,Tuesday,False,False,NaN,NaN,NaN,NaN,NaN,NaN,False,120,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-31 22:00:00,25721.0,1,31,22,0,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1320,Winter
2015-01-31 22:30:00,27309.0,1,31,22,30,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1350,Winter
2015-01-31 23:00:00,26591.0,1,31,23,0,Saturday,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,1380,Winter
